In [1]:
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
import numpy as np
from os.path import join
from copy import deepcopy
# suppress some warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load one local library with additional statistics for the xarray datastructure
import xhydrostats as xs

In [3]:
root = r'/scratch/compound'
t0, t1 = datetime(1980,1,1), datetime(2014,12,30)
q = 95
min_dist = 45
window_size = 7
Npeaks = 50
chunks = {'ensemble':-1, 'time':-1, 'index':250}
drivers = ['compound', 'surge', 'runoff']

In [17]:
glob_attrs = dict(
    title = 'statistics of global compound river mouth reanalysis data',
    source = 'global compound river mouth reanalysis and global tide and surge reanalysis',
    institution = 'Institute for Environmental Studies (IVM) - Vrije Universiteit Amsterdam',
    author = 'Dirk Eilander (dirk.eilander@vu.nl)',
    conventions = "CF-1.7",
    date_created = str(datetime.now().date()),
    history = 'created using xarray v' + xr.__version__,
)

In [8]:
# load regular outputs
fn_gcfr = join(root, 'gcfr.zarr')
fn_out_drivers = join(root, 'cfi_drivers.nc')
ds_cmf = xr.open_zarr(fn_gcfr).sel(time=slice(t0,t1))
ds_cmf.ensemble.data = ds_cmf.ensemble.values.astype(str)

# TODO: merge instant sealvl instead of daily max. fix downstream
ds_sealvl = xr.open_dataset(join(root, 'sealvl_inst.nc')).drop('gtsm_idx').sel(time=slice(t0,t1))
ds_sealvl['sea_surge'] = ds_sealvl['sea_water_level'] - ds_sealvl['sea_water_level_climatology'] 
ds_cmf = xr.merge([ds_sealvl, ds_cmf.drop(['sea_water_level', 'sea_water_level_climatology'])])
ds_cmf['river_discharge_anomaly'] = ds_cmf['river_discharge'] - ds_cmf['river_discharge_climatology'] 

# remove untrusted months -> something funny in gtsm during oct/nov 1990
ds_cmf = ds_cmf.where(ds_cmf.time!=slice(datetime(1990,10,1), datetime(1991,3,1))) #.sel(index=[1617, 1230])

## peaks

In [9]:
fn_peaks = join(root, f'peaks_q{q}d{min_dist}_wdw{window_size}.nc')

In [10]:
da_wl = ds_cmf['water_level'].chunk(chunks)
# peaks (boolean) per scenario
peaks = xr.ufuncs.isfinite(xs.get_peaks(da_wl, min_dist=min_dist, dim='time')).reindex_like(da_wl, fill_value=False).chunk(chunks)
peaks.name = 'peaks_all'
# max water level within +/- 3 days for peaks per scenario
wl_wdwmax = da_wl.rolling(time=window_size, min_periods=1, center=True).construct('window').max('window')
# keep only largest peaks within +/- 3 days between all scen
peaks_max = xr.where(peaks, da_wl>=wl_wdwmax.max('scen'), False).chunk(chunks)
# remove peaks from compound scen where as large as in single scen
peaks_max_n = peaks_max.sum('scen')
peaks_lst = []
for driver in peaks_max.scen.data:
    if driver == 'cmpnd':
        # only compound if compound scenario gives SINGLE largest water level
        peaks_driver = xr.where(np.logical_and(peaks_max_n==1, peaks_max.sel(scen=driver)), True, False)
    else:
        peaks_driver = xr.where(np.logical_and(peaks_max_n>=1, peaks_max.sel(scen=driver)), True, False)
    peaks_lst.append(peaks_driver)
peaks_max = xr.concat(peaks_lst, dim='scen')
# peaks_max['scen'] = peaks_max.scen
peaks_max_n = peaks_max.sum('scen')
assert peaks_max_n.max() == 1

# rank peaks
peaks_val_flat = xr.where(peaks_max, da_wl, -np.inf).max('scen').chunk(chunks)
peaks_ranks_flat = xs.rankdata(-1 * peaks_val_flat, dim='time', method='ordinal') # minus peak values to sort from large to small
peaks_rank = xr.where(peaks_max, peaks_ranks_flat, np.nan)
peaks_rank.name = 'peaks_rank'

# add drivers 
surge_wdw_max = ds_cmf['sea_surge'].rolling(time=window_size, min_periods=1, center=True).construct('window').max('window')
surge_wdw_max.name = 'surge_wdw_max'
runoff_wdw_max = ds_cmf['river_discharge_anomaly'].rolling(time=window_size, min_periods=1, center=True).construct('window').max('window')
runoff_wdw_max.name = 'runoff_wdw_max'

ds_peaks = xr.merge([peaks, peaks_rank, surge_wdw_max, runoff_wdw_max], join='inner').chunk(chunks)
ds_peaks.to_netcdf(fn_peaks)

### peak percentages

In [18]:
# reduce time index
ds_peaks = xr.open_dataset(fn_peaks, chunks=chunks) #.sel(index=[1617, 1230])

# select top Npeaks
peaks_top = ds_peaks[f'peaks_rank'].where(ds_peaks[f'peaks_rank']<=Npeaks)
peaks_top_n = peaks_top.sum('time')
peaks_top_n.name = f'peaks_n'
# calculate statistics for top Npeaks
cmpnd_stats = peaks_top_n.to_dataset()
cmpnd_stats[f'peaks_perc'] =  peaks_top_n / peaks_top_n.sum('scen')
cmpnd_stats[f'peaks_Hmean'] = ds_peaks['surge_wdw_max'].where(peaks_top).mean('time')
cmpnd_stats[f'peaks_Qmean'] = ds_peaks['runoff_wdw_max'].where(peaks_top).mean('time')
# calculate overal statistics
cmpnd_stats[f'Qmean'] = ds_cmf['river_discharge'].mean(['time'])
cmpnd_stats[f'Qmamax'] = ds_cmf['river_discharge'].resample(time = 'A').max('time').mean(['time'])
cmpnd_stats[f'Hmamax'] = ds_cmf['sea_surge'].resample(time = 'A').max('time').mean(['time'])
# close input and save to disk
ds_peaks.close()
fn_out = join(root, f'peaks{Npeaks}_q{q}d{min_dist}_wdw{window_size}_perc.nc')
cmpnd_stats.attrs.update(**glob_attrs)
cmpnd_stats = cmpnd_stats.chunk({k:v for k,v in chunks.items() if k != 'time'})
encoding = {k: {'zlib': True} for k in cmpnd_stats.data_vars}
cmpnd_stats.to_netcdf(fn_out, mode='w', encoding=encoding)

### return levels

In [ ]:
# reduce time index
rp = np.array([1, 2, 5, 10, 20, 30, 34])

ds_peaks = xr.open_dataset(fn_peaks, chunks=chunks) #.sel(index=[1617, 1230])
da_peaks = ds_cmf['water_level'].chunk(chunks).where(ds_peaks['peaks_all'])
wl_extr = xs.return_level(da_peaks, nyears=35, rp=rp)
wl_extr.name = 'return_level'
wl_extr_ci = xs.return_level_ci(da_peaks, nyears=35, rp=rp, alphas=np.array([0.1, 0.25, 0.75, 0.9]), n_samples=1000)
N = ds_peaks['ensemble'].size

ds = wl_extr.to_dataset()
ds['return_level_ci'] = wl_extr_ci
ds['main_driver'] = xr.where(
    wl_extr.sel(scen='runoff').mean('ensemble') > wl_extr.sel(scen='surge').mean('ensemble'), 
    1, # Q 
    -1 # H
) 
wl_extr_single = xr.where(ds['main_driver']==1, wl_extr.sel(scen='runoff'), wl_extr.sel(scen='surge'))
wl_extr_ci_single = xr.where(ds['main_driver']==1, wl_extr_ci.sel(scen='runoff'), wl_extr_ci.sel(scen='surge'))
ds['compound_ratio'] = (wl_extr.sel(scen='cmpnd') - wl_extr_single) / wl_extr_single
ds['compound_ratio_mean'] = ds['compound_ratio'].mean('ensemble')
compound_ratio_std = ds['compound_ratio'].std('ensemble')
compound_ratio_cv = ds['compound_ratio_mean'] / compound_ratio_std
ds['compound_ratio_dir'] =  xr.ufuncs.fabs(xr.ufuncs.sign(ds['compound_ratio']).sum('ensemble')) == N
ds['compound_ratio_sign0'] = xr.ufuncs.fabs(compound_ratio_cv) > (2 / xr.ufuncs.sqrt(N-1))

ds['compound_ratio_sign1'] = xr.where(
    ds['compound_ratio']>0,
    wl_extr_ci.sel(scen='cmpnd', alpha=0.1) >= wl_extr_ci_single.sel(alpha=0.9),
    wl_extr_ci_single.sel(alpha=0.1) >= wl_extr_ci.sel(scen='cmpnd', alpha=0.9),
).sum('ensemble')

# save output to nc
fn_out = join(root, f'peaks_q{q}d{min_dist}_wdw{window_size}_rp.nc')
ds.attrs.update(**glob_attrs)
ds = ds.chunk({k:v for k,v in chunks.items() if k != 'time'})
encoding = {k: {'zlib': True} for k in ds.data_vars}
ds.to_netcdf(fn_out, mode='w', encoding=encoding)


In [ ]:
# save output to nc
# ds_out

## cfi

In [ ]:
# components
da_cmpnd = ds_cmf['water_level'].sel(scen='cmpnd').drop('scen').chunk(chunks)
da_surge = ds_cmf['water_level'].sel(scen='surge').drop('scen').chunk(chunks)
da_runoff = ds_cmf['water_level'].sel(scen='runoff').drop('scen').chunk(chunks)

# compute cfi
da_surge_wdwmax = da_surge.rolling(time=window_size, min_periods=1, center=True).max().chunk(chunks)
da_runoff_wdwmax = da_runoff.rolling(time=window_size, min_periods=1, center=True).max().chunk(chunks)
da_cmpnd_std = da_cmpnd.std('time')
cfi_surge = ((da_cmpnd - da_runoff_wdwmax) / da_cmpnd_std).chunk(chunks)
cfi_surge.name = 'cfi_surge'
cfi_runoff = ((da_cmpnd - da_surge_wdwmax) / da_cmpnd_std).chunk(chunks)
cfi_runoff.name = 'cfi_runoff'
cfi_compound = xr.where(xr.ufuncs.fabs(cfi_runoff)<xr.ufuncs.fabs(cfi_surge), cfi_runoff, cfi_surge)
cfi_compound.name = 'cfi_compound'

# compound water level peaks
threshold = xs.nanpercentile(da_cmpnd, q=q, dim='time')
peaks = xr.ufuncs.isfinite(xs.peaks_over_threshold(da_cmpnd, min_dist=min_dist, threshold=threshold, dim='time', chunks=chunks))
peaks = peaks.reindex_like(cfi_compound, fill_value=False).chunk(chunks)
peaks.name = 'peaks_all'
# classify peaks
peaks_compound = xr.where(peaks, np.logical_or(np.logical_and(cfi_surge>0,  cfi_runoff>0),
                                               np.logical_and(cfi_surge<0,  cfi_runoff<0)), False)
peaks_compound.name = 'peaks_compound'
peaks_surge    = xr.where(peaks, np.logical_and(cfi_surge>=0,  cfi_runoff<=0), False)
peaks_surge.name = 'peaks_surge'
peaks_runoff   = xr.where(peaks, np.logical_and(cfi_surge<=0, cfi_runoff>=0), False)
peaks_runoff.name = 'peaks_runoff'

# add drivers 
surge_driver = ds_cmf['sea_surge'].rolling(time=window_size, min_periods=1, center=True).max()
surge_driver.name = 'surge_wdw_max'
runoff_driver = ds_cmf['river_discharge'].rolling(time=window_size, min_periods=1, center=True).max()
runoff_driver.name = 'runoff_wdw_max'

ds_cfi = xr.merge([
    cfi_runoff, cfi_surge, cfi_compound,
    surge_driver, runoff_driver,
    peaks_runoff, peaks_surge, peaks_compound, peaks,
], join='inner').chunk(chunks)
ds_cfi.to_netcdf(join(root, f'cfi_q{q}d{min_dist}_wdw{window_size}.nc'))
# ds_cfi.compute()

In [ ]:
fn_out = join(root, f'cfi_q{q}d{min_dist}_wdw{window_size}_stats.nc')
fn_cfi = join(root, f'cfi_q{q}d{min_dist}_wdw{window_size}.nc')

ds_cfi = xr.open_dataset(fn_cfi, chunks=chunks)
drivers = ['compound', 'surge', 'runoff']

peaks_all_n = ds_cfi['peaks_all'].sum('time')
peaks_all_n.name = 'peaks_all_n'
cmpnd_stats = peaks_all_n.to_dataset()
for driver in drivers:
    peaks_driver = ds_cfi[f'peaks_{driver}']
    cfi_driver = ds_cfi[f'cfi_{driver}']
    cfi_driver_peaks = cfi_driver.where(peaks_driver)
    # percentage of peaks
    peaks_driver_n = peaks_driver.sum('time') 
    cmpnd_stats[f'peaks_{driver}_n'] = peaks_driver_n
    cmpnd_stats[f'peaks_{driver}_perc'] = peaks_driver_n / peaks_all_n
    # cfi > 1 std 
    cmpnd_stats[f'peaks_{driver}_n_cfi1'] = xr.where(cfi_driver_peaks>1, 1, 0).sum('time')
    cmpnd_stats[f'peaks_{driver}_n_cfi2'] = xr.where(cfi_driver_peaks>2, 1, 0).sum('time')
    # cfi stats
    cmpnd_stats[f'cfi_{driver}'] = cfi_driver_peaks.mean(dim='time')
    cmpnd_stats[f'cfi_{driver}_std'] = cfi_driver_peaks.std(dim='time')
    # CV with abs mean
    cmpnd_stats[f'cfi_{driver}_cv'] = xr.ufuncs.fabs(cmpnd_stats[f'cfi_{driver}']) / cmpnd_stats[f'cfi_{driver}_std']
    # CV > 2 / sqrt(N-1)
    cmpnd_stats[f'cfi_{driver}_sign'] = (cmpnd_stats[f'cfi_{driver}_cv'] > (2 / xr.ufuncs.sqrt(peaks_driver_n-1))) * np.sign(cmpnd_stats[f'cfi_{driver}'])
    
cmpnd_stats[f'river_discharge_mean'] = ds_cmf['river_discharge'].mean(['time'])
cmpnd_stats[f'river_discharge_mean_amax'] = ds_cmf['river_discharge'].resample(time = 'A').max('time').mean(['time'])
cmpnd_stats[f'sea_surge_mean_amax'] = ds_cmf['sea_surge'].resample(time = 'A').max('time').mean(['time'])

peaks_test_n = (cmpnd_stats['peaks_compound_n'] + cmpnd_stats['peaks_runoff_n'] + cmpnd_stats['peaks_surge_n'])
assert np.all(cmpnd_stats['peaks_all_n'] == peaks_test_n ).compute().values

datasets = [cmpnd_stats]

In [ ]:
for driver in drivers:       
    cfi_peaks = ds_cfi[f'cfi_{driver}'].where(ds_cfi[f'peaks_{driver}'])
    # timing of cfi
    cmpnd_temp_stats = xs.mean_flood_day_stats(cfi_peaks.fillna(-np.inf))
    cmpnd_temp_stats = cmpnd_temp_stats.rename({k:f'cfi_{driver}_{k}' for k in cmpnd_temp_stats.data_vars})
    datasets.append(cmpnd_temp_stats)

In [ ]:
# save output to nc

ds_out = xr.merge(datasets)
ds_out.attrs = dict(
    title = 'statistics of global compound river mouth reanalysis data',
    source = 'global compound river mouth reanalysis and global tide and surge reanalysis',
    institution = 'Institute for Environmental Studies (IVM) - Vrije Universiteit Amsterdam',
    author = 'Dirk Eilander (dirk.eilander@vu.nl)',
    conventions = "CF-1.7",
    date_created = str(datetime.now().date()),
    history = 'created using xarray v' + xr.__version__,
)

ds_out = ds_out.chunk({k:v for k,v in chunks.items() if k != 'time'})
encoding = {k: {'zlib': True} for k in ds_out.data_vars}
ds_out.to_netcdf(fn_out, mode='w', encoding=encoding)
# ds_out

In [ ]:
# reduce index/space dimension

fn_out = join(root, f'cfi_q{q}d{min_dist}_wdw{window_size}_stats2.csv')
fn_cfi_stats = join(root, f'cfi_q{q}d{min_dist}_wdw{window_size}_stats.nc')

cmpnd_stats = xr.open_dataset(fn_cfi_stats, chunks={'index': -1, 'ensemble':-1}).drop('percentile')
drivers = ['compound', 'surge', 'runoff']

peaks_all_n = cmpnd_stats['peaks_all_n'].sum('index')
peaks_all_n.name = 'peaks_all_n'
cmpnd_stats2 = peaks_all_n.to_dataset()
for driver in drivers:
    cfi_driver = cmpnd_stats[f'cfi_{driver}']
    cmpnd_stats2[f'peaks_{driver}_perc'] = cmpnd_stats[f'peaks_{driver}_n'].sum('index') / peaks_all_n
    cmpnd_stats2[f'peaks_{driver}_perc_cfi1'] = cmpnd_stats[f'peaks_{driver}_n_cfi1'].sum('index') / peaks_all_n
    cmpnd_stats2[f'peaks_{driver}_perc_cfi2'] = cmpnd_stats[f'peaks_{driver}_n_cfi2'].sum('index') / peaks_all_n
    cmpnd_stats2[f'cfi_{driver}_min'] = cfi_driver.min('index') 
    cmpnd_stats2[f'cfi_{driver}_max'] = cfi_driver.max('index')
    cfi_q = xs.nanpercentile(cfi_driver, [5, 50 ,95], dim='index')
    cmpnd_stats2[f'cfi_{driver}_med'] = cfi_q.sel(percentile=50).drop('percentile')
    cmpnd_stats2[f'cfi_{driver}_p05'] = cfi_q.sel(percentile=5).drop('percentile')
    cmpnd_stats2[f'cfi_{driver}_p95'] = cfi_q.sel(percentile=95).drop('percentile')


cmpnd_stats2.to_dataframe().T.to_csv(fn_out, float_format='%.4f')

## plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1,figsize=(10,6))

sel = dict(time=slice('01-01-1995', '09-01-1996'))
sel = dict(time=slice('01-01-1991', '09-01-1991'))
da_wl.isel(index=0, ensemble=1).sel(**sel).plot.line(x='time', ax=ax)
xr.where(np.logical_and(peaks100, peaks.sum('scen')==2), peaks_val, np.nan).isel(index=0, ensemble=1).sel(**sel).to_series().plot(ax=ax, linewidth=0, markersize=10, marker='x')

fig2, ax2 = plt.subplots(1,1,figsize=(10,6))
# ds_cmf['river_discharge'].isel(index=0, ensemble=1).sel(**sel).plot.line(x='time', ax=ax2)
ds_cmf['sea_surge'].isel(index=0).sel(**sel).plot.line(x='time', ax=ax2)
# ds_cmf['sea_water_level_climatology'].isel(index=0).sel(**sel).plot.line(x='time', ax=ax2)

In [ ]:
%matplotlib inline
driver = 'compound'
cfi_driver = cmpnd_stats[f'peaks_{driver}_perc']*100
cfi_driver.to_series().unstack(-1).T.plot.box(whis=[0,95], showfliers=False)

In [ ]:
driver = 'compound'
cfi_driver = cmpnd_stats[f'cfi_{driver}']
cfi_driver.to_series().unstack(-1).T.plot.box(whis=[0,95], showfliers=False)

In [ ]:
# test for single station
%matplotlib inline
import matplotlib.pyplot as plt
q = 95
min_dist = 30
rid = 387
# rid = 3200


ds_i= ds_cmf.isel(index=rid).sel(ensemble='nerc').squeeze().load()

# components
da_cmpnd = ds_i['water_level'].sel(scen='cmpnd') 
da_surge = ds_i['water_level'].sel(scen='surge')
da_runoff = ds_i['water_level'].sel(scen='runoff')
da_surge_wdwmax = da_surge.rolling(time=7, min_periods=1, center=True).max()
da_runoff_wdwmax = da_runoff.rolling(time=7, min_periods=1, center=True).max()
da_cmpnd_std = da_cmpnd.std('time')
cfi_surge = ((da_cmpnd - da_runoff_wdwmax) / da_cmpnd_std)
cfi_runoff = ((da_cmpnd - da_surge_wdwmax) / da_cmpnd_std)
cfi_runoff

threshold = xs.nanpercentile(da_cmpnd, q=q, dim='time')
ts = xs.peaks_over_threshold(da_cmpnd, min_dist=min_dist, threshold=threshold, dim='time') 
cfi_surge_peaks = cfi_surge.where(xr.ufuncs.isfinite(ts))
cfi_runoff_peaks = cfi_runoff.where(xr.ufuncs.isfinite(ts))

fig, ((ax1, ax2, ax3), (ax12, ax22, ax32)) = plt.subplots(2, 3, figsize=(15, 9), sharex=True)
sel = dict(time=slice('07-01-2002', '07-01-2003'))
ymax = max(ds_i['water_level'].sel(**sel).max().values, ds_i['sea_water_level'].sel(**sel).max().values)
ymin = min(ds_i['water_level'].sel(**sel).min().values, ds_i['sea_water_level'].sel(**sel).min().values)
ylim = [ymin, ymax]
ylab = 'water surface elevation [m+EGM96]'

# da_cmpnd.sel(**sel).plot(ax=ax2, c='red')
ds_i['water_level'].sel(**sel, scen='surge').plot(ax=ax2, x='time', label='surge', c='blue')
ds_i['water_level'].sel(**sel, scen='runoff').plot(ax=ax2, x='time', label='runoff', c='green')
ds_i['water_level'].sel(**sel, scen='cmpnd').plot(ax=ax2, x='time', label='compound', c='red')
cfi_surge.sel(**sel).plot(ax=ax22, c='blue', linestyle='--')
cfi_runoff.sel(**sel).plot(ax=ax22, c='green', linestyle='--')
ax2.set_title('river mouth')
ax22.set_title('')
ax2.set_ylabel(ylab)
ax2.set_ylim(ylim)
ax2.legend()


ds_i['river_discharge'].sel(**sel).plot(ax=ax3, c='green')
ds_i['river_discharge_climatology'].sel(**sel).plot(ax=ax3, c='green', linestyle='--')

ax3.set_title('upstream boundary')
ax3.set_ylabel('discharge [m3/s]')

ds_i['sea_water_level_climatology'].sel(**sel).plot(ax=ax1, c='blue', linestyle='--')
ds_i['sea_water_level'].sel(**sel).plot(ax=ax1, c='blue')
ax1.set_title('downstream boundary')
ax1.set_ylabel(ylab)
ax1.set_ylim(ylim)

# ds_i['sea_water_level_climatology'].sel(**sel).plot(ax=ax1, c='magenta', x='time')

# if ts.sel(**sel).dropna('time').values.size>1: # needs two dates to plot
#     ts.sel(**sel).dropna('time').plot(c='r', marker='*', markersize=9, linewidth=0, ax=ax)
#     cfi_runoff_peaks.sel(**sel).dropna('time').plot(c='r', marker='*', markersize=9, linewidth=0, ax=ax2)


# cfi_peaks.mean('time').compute().values, ds_i.to_series().unstack(0).tail(5)